In [ ]:
# 5_Submission_AE_stats_valid (Kaggle)
# Goal: generate a submit-ready submission.csv on Kaggle (AE version, cfcs top4 valid)
#
# Place these files in the Kaggle Dataset or Notebook input:
# - Data/Selection/selected_features_ae_{AE_TAG}.parquet
# - Data/Selection/ae_input_features_{AE_TAG}.parquet
# - Data/Selection/ae_model_{AE_TAG}.pth
# - Data/Selection/ae_scaler_{AE_TAG}.joblib
#
# This notebook will:
# - Read the competition raw CSV
# - Reproduce Processing + Feature Engineering
# - Compute AE features (z1/z2/z3 + recon_error)
# - Subset columns using the uploaded feature list
# - Export submission.csv (climate_risk_* + required fields only)

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import joblib

import torch
import torch.nn as nn

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

RISK_CATEGORIES = ["heat_stress", "unseasonably_cold", "excess_precip", "drought"]

DATA_DIR = Path("/kaggle/input/forecasting-the-future-the-helios-corn-climate-challenge")
WORK_DIR = Path("/kaggle/working")
INPUT_DIR = Path("/kaggle/input/reproduce-ae-cfcs-0-5-top4-2-valid")

RAW_MAIN = DATA_DIR / "corn_climate_risk_futures_daily_master.csv"
RAW_SHARE = DATA_DIR / "corn_regional_market_share.csv"

# Select AE model and feature files
SIGNIFICANCE_THRESHOLD = 0.5
THRESH_TAG_2 = f"{SIGNIFICANCE_THRESHOLD:.2f}".rstrip("0").rstrip(".")
TAG_2 = f"cfcs_{THRESH_TAG_2}_top4_2"
AE_TAG = f"ae_{TAG_2}_valid"


def resolve_input_path(filename: str) -> Path:
    direct = INPUT_DIR / filename
    if direct.exists():
        return direct
    # Try to auto-find under /kaggle/input
    matches = list(Path("/kaggle/input").rglob(filename))
    if matches:
        return matches[0]
    return direct


FEAT_PATH = resolve_input_path(f"selected_features_ae_{AE_TAG}.parquet")
AE_INPUT_PATH = resolve_input_path(f"ae_input_features_{AE_TAG}.parquet")
AE_MODEL_PATH = resolve_input_path(f"ae_model_{AE_TAG}.pth")
AE_SCALER_PATH = resolve_input_path(f"ae_scaler_{AE_TAG}.joblib")

print("FEAT_PATH:", FEAT_PATH)
print("AE_INPUT_PATH:", AE_INPUT_PATH)
print("AE_MODEL_PATH:", AE_MODEL_PATH)
print("AE_SCALER_PATH:", AE_SCALER_PATH)


assert RAW_MAIN.exists()
assert RAW_SHARE.exists()
assert FEAT_PATH.exists()
assert AE_INPUT_PATH.exists()
assert AE_MODEL_PATH.exists()
assert AE_SCALER_PATH.exists()


class AutoEncoder(nn.Module):
    def __init__(self, input_dim: int, latent_dim: int = 3):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim),
        )

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat, z


FEAT_PATH: /kaggle/input/reproduce-ae-cfcs-0-5-top4-2-valid/selected_features_ae_ae_cfcs_0.5_top4_2_valid.parquet
AE_INPUT_PATH: /kaggle/input/reproduce-ae-cfcs-0-5-top4-2-valid/ae_input_features_ae_cfcs_0.5_top4_2_valid.parquet
AE_MODEL_PATH: /kaggle/input/reproduce-ae-cfcs-0-5-top4-2-valid/ae_model_ae_cfcs_0.5_top4_2_valid.pth
AE_SCALER_PATH: /kaggle/input/reproduce-ae-cfcs-0-5-top4-2-valid/ae_scaler_ae_cfcs_0.5_top4_2_valid.joblib


In [ ]:
# Load raw data

df = pd.read_csv(RAW_MAIN, low_memory=False)
df["date_on"] = pd.to_datetime(df["date_on"], errors="coerce")
share = pd.read_csv(RAW_SHARE, low_memory=False)

print("raw:", df.shape)

# Processing: ffill + dropna(futures)
climate_cols = [c for c in df.columns if c.startswith("climate_risk_")]
futures_cols = [c for c in df.columns if c.startswith("futures_")]

ffill_cols = [c for c in (climate_cols + futures_cols) if c not in ["ID", "region_id", "date_on"]]

df = df.sort_values(["region_id", "date_on"]).copy()
df[ffill_cols] = df.groupby("region_id")[ffill_cols].ffill()
df[climate_cols] = df[climate_cols].fillna(0)

df = df.dropna(subset=futures_cols).copy()

print("after processing:", df.shape)



raw: (320661, 41)
after processing: (320447, 41)


In [ ]:
# Feature Engineering

df = df.merge(share[["region_id", "percent_country_production"]], on="region_id", how="left")
df["percent_country_production"] = df["percent_country_production"].fillna(1.0)


def build_features(base: pd.DataFrame, seasonal_stats: pd.DataFrame | None = None, thresholds: dict | None = None):
    df = base.copy()
    new_cols: list[str] = []

    # Collect features first, then concatenate once to avoid DataFrame fragmentation
    feats: dict[str, pd.Series] = {}

    # Time features
    feats["climate_risk_day_of_year"] = df["date_on"].dt.dayofyear
    feats["climate_risk_quarter"] = df["date_on"].dt.quarter
    feats["climate_risk_season_sin"] = np.sin(2 * np.pi * feats["climate_risk_day_of_year"] / 365.0)
    feats["climate_risk_season_cos"] = np.cos(2 * np.pi * feats["climate_risk_day_of_year"] / 365.0)

    # Hemisphere-aligned seasonal features & key window weights
    south_countries = {
        "Brazil",
        "Argentina",
        "Australia",
        "South Africa",
        "Uruguay",
        "Paraguay",
        "Bolivia",
        "Chile",
        "Peru",
        "New Zealand",
    }
    is_south = df["country_name"].isin(south_countries)
    month = df["date_on"].dt.month
    month_adj = np.where(is_south, ((month + 5) % 12) + 1, month)
    season_weight = np.where(
        np.isin(month_adj, [7, 8]),
        1.5,
        np.where(np.isin(month_adj, [6, 9]), 1.2, 1.0),
    )
    feats["climate_risk_season_weight"] = season_weight

    doy_shift = np.where(is_south, 182, 0)
    doy_shifted = (feats["climate_risk_day_of_year"] + doy_shift) % 365
    feats["climate_risk_season_sin_shift"] = np.sin(2 * np.pi * doy_shifted / 365.0)
    feats["climate_risk_season_cos_shift"] = np.cos(2 * np.pi * doy_shifted / 365.0)

    # Base risk scores + production weighting
    for risk_type in RISK_CATEGORIES:
        low_col = f"climate_risk_cnt_locations_{risk_type}_risk_low"
        med_col = f"climate_risk_cnt_locations_{risk_type}_risk_medium"
        high_col = f"climate_risk_cnt_locations_{risk_type}_risk_high"

        total = df[low_col] + df[med_col] + df[high_col]
        score = (df[med_col] + 2.0 * df[high_col]) / (total + 1e-6)
        high_share = df[high_col] / (total + 1e-6)
        med_share = df[med_col] / (total + 1e-6)
        low_share = df[low_col] / (total + 1e-6)
        severity_051 = (0.5 * df[med_col] + 1.0 * df[high_col]) / (total + 1e-6)
        balance = (df[high_col] - df[low_col]) / (total + 1e-6)
        entropy = -(
            low_share * np.log(low_share + 1e-6)
            + med_share * np.log(med_share + 1e-6)
            + high_share * np.log(high_share + 1e-6)
        )

        weight = df["percent_country_production"] / 100.0
        weighted = score * weight
        weighted_high = high_share * weight
        weighted_med = med_share * weight
        weighted_sev051 = severity_051 * weight
        weighted_balance = balance * weight

        feats[f"climate_risk_{risk_type}_score"] = score
        feats[f"climate_risk_{risk_type}_weighted"] = weighted
        feats[f"climate_risk_{risk_type}_high_share"] = high_share
        feats[f"climate_risk_{risk_type}_med_share"] = med_share
        feats[f"climate_risk_{risk_type}_low_share"] = low_share
        feats[f"climate_risk_{risk_type}_severity_051"] = severity_051
        feats[f"climate_risk_{risk_type}_balance"] = balance
        feats[f"climate_risk_{risk_type}_entropy"] = entropy
        feats[f"climate_risk_{risk_type}_weighted_high_share"] = weighted_high
        feats[f"climate_risk_{risk_type}_weighted_med_share"] = weighted_med
        feats[f"climate_risk_{risk_type}_weighted_severity_051"] = weighted_sev051
        feats[f"climate_risk_{risk_type}_weighted_balance"] = weighted_balance

    # Concatenate base features once
    base_feat_df = pd.DataFrame(feats, index=df.index)
    df = df.join(base_feat_df)
    new_cols.extend(list(base_feat_df.columns))

    # Sort for time-series features (by region_id)
    df = df.sort_values(["region_id", "date_on"]).copy()

    # Rolling / Lag / EMA / Momentum
    roll_feats: dict[str, pd.Series] = {}
    ROLL_WINDOWS = [7, 14, 30, 60, 90]
    LAGS = [1, 7, 14, 30]
    EMAS = [7, 14, 30, 60]

    for window in ROLL_WINDOWS:
        for risk_type in RISK_CATEGORIES:
            score_col = f"climate_risk_{risk_type}_score"

            roll_feats[f"climate_risk_{risk_type}_ma_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
            roll_feats[f"climate_risk_{risk_type}_max_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=1).max())
            )
            roll_feats[f"climate_risk_{risk_type}_vol_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=2).std())
            )
            roll_feats[f"climate_risk_{risk_type}_cumsum_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=1).sum())
            )

    for lag in LAGS:
        for risk_type in RISK_CATEGORIES:
            score_col = f"climate_risk_{risk_type}_score"
            roll_feats[f"climate_risk_{risk_type}_lag_{lag}d"] = df.groupby("region_id")[score_col].shift(lag)

    for span in EMAS:
        for risk_type in RISK_CATEGORIES:
            score_col = f"climate_risk_{risk_type}_score"
            roll_feats[f"climate_risk_{risk_type}_ema_{span}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.ewm(span=span, min_periods=1).mean())
            )

    # Momentum / Acceleration
    for risk_type in RISK_CATEGORIES:
        score_col = f"climate_risk_{risk_type}_score"
        c1 = df.groupby("region_id")[score_col].diff(1)
        roll_feats[f"climate_risk_{risk_type}_change_1d"] = c1
        roll_feats[f"climate_risk_{risk_type}_change_7d"] = df.groupby("region_id")[score_col].diff(7)
        roll_feats[f"climate_risk_{risk_type}_acceleration"] = c1.groupby(df["region_id"]).diff(1)

        lag7 = df.groupby("region_id")[score_col].shift(7)
        roll_feats[f"climate_risk_{risk_type}_roc_7d"] = (df[score_col] - lag7) / (lag7 + 1e-6)

    # Event and persistence features
    EVENT_TAUS = [0.2, 0.3]
    EVENT_WINDOWS = [7, 14, 30]
    SPIKE_WINDOW = 30
    SPIKE_Z = 2.0

    for risk_type in RISK_CATEGORIES:
        hs_col = f"climate_risk_{risk_type}_high_share"

        for tau in EVENT_TAUS:
            runlen = df.groupby("region_id")[hs_col].transform(
                lambda s: (s > tau).groupby((~(s > tau)).cumsum()).cumsum()
            )
            roll_feats[f"climate_risk_{risk_type}_high_runlen_{int(tau*100)}"] = runlen

            time_since = df.groupby("region_id")[hs_col].transform(
                lambda s: (~(s > tau)).groupby((s > tau).cumsum()).cumcount()
            )
            roll_feats[f"climate_risk_{risk_type}_time_since_high_{int(tau*100)}"] = time_since

            for window in EVENT_WINDOWS:
                auc = df.groupby("region_id")[hs_col].transform(
                    lambda s: (s - tau).clip(lower=0).rolling(window, min_periods=1).sum()
                )
                roll_feats[f"climate_risk_{risk_type}_event_auc_{int(tau*100)}_{window}d"] = auc

        mean_ = df.groupby("region_id")[hs_col].transform(lambda s: s.rolling(SPIKE_WINDOW, min_periods=2).mean())
        std_ = df.groupby("region_id")[hs_col].transform(lambda s: s.rolling(SPIKE_WINDOW, min_periods=2).std())
        roll_feats[f"climate_risk_{risk_type}_spike_{SPIKE_WINDOW}d"] = (
            (df[hs_col] - mean_) > (SPIKE_Z * std_)
        ).astype(int)

    roll_feat_df = pd.DataFrame(roll_feats, index=df.index)
    df = df.join(roll_feat_df)
    new_cols.extend(list(roll_feat_df.columns))

    # Non-linear & Combination
    combo_feats: dict[str, pd.Series] = {}
    score_cols = [f"climate_risk_{r}_score" for r in RISK_CATEGORIES]
    weighted_cols = [f"climate_risk_{r}_weighted" for r in RISK_CATEGORIES]

    combo_feats["climate_risk_temperature_stress"] = df[["climate_risk_heat_stress_score", "climate_risk_unseasonably_cold_score"]].max(axis=1)
    combo_feats["climate_risk_precipitation_stress"] = df[["climate_risk_excess_precip_score", "climate_risk_drought_score"]].max(axis=1)
    combo_feats["climate_risk_overall_stress"] = df[score_cols].max(axis=1)
    combo_feats["climate_risk_combined_stress"] = df[score_cols].mean(axis=1)

    combo_feats["climate_risk_temperature_stress_weighted"] = df[["climate_risk_heat_stress_weighted", "climate_risk_unseasonably_cold_weighted"]].max(axis=1)
    combo_feats["climate_risk_precipitation_stress_weighted"] = df[["climate_risk_excess_precip_weighted", "climate_risk_drought_weighted"]].max(axis=1)
    combo_feats["climate_risk_overall_weighted"] = df[weighted_cols].max(axis=1)
    combo_feats["climate_risk_combined_weighted"] = df[weighted_cols].mean(axis=1)

    combo_feats["climate_risk_precip_drought_diff"] = df["climate_risk_excess_precip_score"] - df["climate_risk_drought_score"]
    combo_feats["climate_risk_temp_diff"] = df["climate_risk_heat_stress_score"] - df["climate_risk_unseasonably_cold_score"]
    combo_feats["climate_risk_precip_drought_ratio"] = df["climate_risk_excess_precip_score"] / (df["climate_risk_drought_score"] + 0.01)

    combo_feats["climate_risk_heat_cold_sum"] = df["climate_risk_heat_stress_score"] + df["climate_risk_unseasonably_cold_score"]
    combo_feats["climate_risk_drought_excess_sum"] = df["climate_risk_drought_score"] + df["climate_risk_excess_precip_score"]
    combo_feats["climate_risk_heat_drought_interact"] = df["climate_risk_heat_stress_score"] * df["climate_risk_drought_score"]
    combo_feats["climate_risk_cold_excess_interact"] = df["climate_risk_unseasonably_cold_score"] * df["climate_risk_excess_precip_score"]

    # season/time modulation
    for risk_type in RISK_CATEGORIES:
        score_col = f"climate_risk_{risk_type}_score"
        weighted_col = f"climate_risk_{risk_type}_weighted"
        combo_feats[f"climate_risk_{risk_type}_score_season_sin"] = df[score_col] * df["climate_risk_season_sin"]
        combo_feats[f"climate_risk_{risk_type}_score_season_cos"] = df[score_col] * df["climate_risk_season_cos"]
        combo_feats[f"climate_risk_{risk_type}_weighted_season_sin"] = df[weighted_col] * df["climate_risk_season_sin"]
        combo_feats[f"climate_risk_{risk_type}_weighted_season_cos"] = df[weighted_col] * df["climate_risk_season_cos"]
        combo_feats[f"climate_risk_{risk_type}_score_qtr"] = df[score_col] * df["climate_risk_quarter"]
        combo_feats[f"climate_risk_{risk_type}_score_doy"] = df[score_col] * (df["climate_risk_day_of_year"] / 365.0)
        combo_feats[f"climate_risk_{risk_type}_season_w"] = df[score_col] * df["climate_risk_season_weight"]

    combo_df = pd.DataFrame(combo_feats, index=df.index)
    df = df.join(combo_df)
    new_cols.extend(list(combo_df.columns))

    # Seasonal anomalies (z-score by country_name + month)
    df["month"] = df["date_on"].dt.month
    if seasonal_stats is None:
        seasonal_stats = (
            df.groupby(["country_name", "month"])
            .agg(**{f"{r}_mean": (f"climate_risk_{r}_score", "mean") for r in RISK_CATEGORIES},
                 **{f"{r}_std": (f"climate_risk_{r}_score", "std") for r in RISK_CATEGORIES})
            .reset_index()
        )

    df = df.merge(seasonal_stats, on=["country_name", "month"], how="left")

    z_feats: dict[str, pd.Series] = {}
    for r in RISK_CATEGORIES:
        mean_c = f"{r}_mean"
        std_c = f"{r}_std"
        z_feats[f"climate_risk_{r}_seasonal_z"] = (df[f"climate_risk_{r}_score"] - df[mean_c]) / (df[std_c] + 1e-6)

    z_df = pd.DataFrame(z_feats, index=df.index)
    df = df.join(z_df)
    new_cols.extend(list(z_df.columns))

    # Extreme risk thresholds (computed on full data)
    if thresholds is None:
        thresholds = {}
        for r in RISK_CATEGORIES:
            thresholds[r] = df[f"climate_risk_{r}_high_share"].quantile(0.9)

    flag_feats: dict[str, pd.Series] = {}
    for r in RISK_CATEGORIES:
        thr = thresholds[r]
        flag_feats[f"climate_risk_{r}_extreme"] = (df[f"climate_risk_{r}_high_share"] >= thr).astype(int)

    flag_df = pd.DataFrame(flag_feats, index=df.index)
    df = df.join(flag_df)
    new_cols.extend(list(flag_df.columns))

    # NaN 处理（一次性）
    df[new_cols] = df[new_cols].fillna(0)

    return df, seasonal_stats, thresholds, new_cols


def add_country_agg(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    country_feats = []
    for risk_type in RISK_CATEGORIES:
        score_col = f"climate_risk_{risk_type}_score"
        weighted_col = f"climate_risk_{risk_type}_weighted"

        country_agg = (
            df.groupby(["country_name", "date_on"])
            .agg(
                **{
                    f"climate_risk_country_{risk_type}_score_mean": (score_col, "mean"),
                    f"climate_risk_country_{risk_type}_score_max": (score_col, "max"),
                    f"climate_risk_country_{risk_type}_score_std": (score_col, "std"),
                    f"climate_risk_country_{risk_type}_weighted_sum": (weighted_col, "sum"),
                }
            )
            .reset_index()
        )

        base_cols = [c for c in country_agg.columns if c not in ["country_name", "date_on"]]
        country_feats.extend(base_cols)
        df = df.merge(country_agg, on=["country_name", "date_on"], how="left")

        high_share_col = f"climate_risk_{risk_type}_high_share"

        def _country_metrics(g: pd.DataFrame) -> pd.Series:
            w = g["percent_country_production"].fillna(0) / 100.0
            x = g[score_col]
            hs = g[high_share_col]
            w_sum = w.sum()

            if w_sum > 0:
                w_mean = np.average(x, weights=w)
                w_std = np.sqrt(np.average((x - w_mean) ** 2, weights=w))
            else:
                w_mean = x.mean()
                w_std = x.std(ddof=0)

            s = w * x
            denom = (s.sum() ** 2) + 1e-6
            hhi = (s ** 2).sum() / denom
            exposed = (w * (hs > 0.3)).sum()

            if s.size <= 1:
                top_diff = float(s.max()) if s.size == 1 else 0.0
            else:
                s_sorted = np.sort(s.to_numpy())
                top_diff = float(s_sorted[-1] - s_sorted[-2])

            return pd.Series(
                {
                    f"climate_risk_country_{risk_type}_score_wmean": w_mean,
                    f"climate_risk_country_{risk_type}_score_wstd": w_std,
                    f"climate_risk_country_{risk_type}_concentration_hhi": hhi,
                    f"climate_risk_country_{risk_type}_prod_exposed_share": exposed,
                    f"climate_risk_country_{risk_type}_top1_minus_top2": top_diff,
                }
            )

        extra = (
            df.groupby(["country_name", "date_on"], sort=False)
            .apply(_country_metrics)
            .reset_index()
        )

        extra_cols = [c for c in extra.columns if c not in ["country_name", "date_on"]]
        country_feats.extend(extra_cols)
        df = df.merge(extra, on=["country_name", "date_on"], how="left")

    # NaN fill
    df[country_feats] = df[country_feats].fillna(0)
    return df, country_feats


df, seasonal_stats, thresholds, feat_cols = build_features(df, seasonal_stats=None, thresholds=None)
# Country aggregation (computed on full data)
df, country_cols = add_country_agg(df)

print("engineered:", len(feat_cols) + len(country_cols), "| df:", df.shape)

# === Generate AE features ===
ae_input_features = pd.read_parquet(AE_INPUT_PATH)["feature"].tolist()
missing = [c for c in ae_input_features if c not in df.columns]
if missing:
    raise ValueError(f"AE 输入列缺失: {missing[:5]} ... total={len(missing)}")

ae_df = df[ae_input_features].fillna(0.0)
scaler = joblib.load(AE_SCALER_PATH)
X_scaled = scaler.transform(ae_df)

model = AutoEncoder(input_dim=X_scaled.shape[1], latent_dim=3)
model.load_state_dict(torch.load(AE_MODEL_PATH, map_location="cpu"))
model.eval()

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
with torch.no_grad():
    X_hat, Z = model(X_tensor)

Z = Z.numpy()

# latent
for i in range(3):
    df[f"climate_risk_ae_z{i+1}"] = Z[:, i]

# recon error
recon_error = ((X_hat - X_tensor) ** 2).mean(dim=1).numpy()
df["climate_risk_ae_recon_error"] = recon_error

/tmp/ipykernel_17/773190809.py:321: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_country_metrics)
/tmp/ipykernel_17/773190809.py:321: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_country_metrics)
/tmp/ipykernel_17/773190809.py:321: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns

engineered: 314 | df: (320447, 365)


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
selected_feats = pd.read_parquet(FEAT_PATH)["feature"].tolist()
selected_feats = [c for c in selected_feats if c in df.columns]

print("selected_feats:", len(selected_feats))

selected_feats: 4


In [6]:
selected_feats

['climate_risk_ae_z1',
 'climate_risk_ae_z2',
 'climate_risk_ae_z3',
 'climate_risk_ae_recon_error']

In [ ]:
# === Align Kaggle valid samples (critical) ===
# Replicate host notebook's valid_ids logic: based on minimum feature engineering dropna()

_tmp = pd.read_csv(RAW_MAIN, low_memory=False)
_tmp["date_on"] = pd.to_datetime(_tmp["date_on"], errors="coerce")
_tmp = _tmp.merge(share[["region_id", "percent_country_production"]], on="region_id", how="left")
_tmp["percent_country_production"] = _tmp["percent_country_production"].fillna(1.0)

# Base risk scores
for risk_type in RISK_CATEGORIES:
    low = f"climate_risk_cnt_locations_{risk_type}_risk_low"
    med = f"climate_risk_cnt_locations_{risk_type}_risk_medium"
    high = f"climate_risk_cnt_locations_{risk_type}_risk_high"
    total = _tmp[low] + _tmp[med] + _tmp[high]
    score = (_tmp[med] + 2 * _tmp[high]) / (total + 1e-6)
    weighted = score * (_tmp["percent_country_production"] / 100.0)
    _tmp[f"climate_risk_{risk_type}_score"] = score
    _tmp[f"climate_risk_{risk_type}_weighted"] = weighted

# Interactions (和 host notebook 一致)
score_cols = [f"climate_risk_{r}_score" for r in RISK_CATEGORIES]
_tmp["climate_risk_temperature_stress"] = _tmp[["climate_risk_heat_stress_score", "climate_risk_unseasonably_cold_score"]].max(axis=1)
_tmp["climate_risk_precipitation_stress"] = _tmp[["climate_risk_excess_precip_score", "climate_risk_drought_score"]].max(axis=1)
_tmp["climate_risk_overall_stress"] = _tmp[score_cols].max(axis=1)
_tmp["climate_risk_combined_stress"] = _tmp[score_cols].mean(axis=1)

# Sort for time-series ops
_tmp = _tmp.sort_values(["region_id", "date_on"]).copy()

# Rolling MA / Max (7/14/30)
for window in [7, 14, 30]:
    for risk_type in RISK_CATEGORIES:
        score_col = f"climate_risk_{risk_type}_score"
        _tmp[f"climate_risk_{risk_type}_ma_{window}d"] = _tmp.groupby("region_id")[score_col].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
        _tmp[f"climate_risk_{risk_type}_max_{window}d"] = _tmp.groupby("region_id")[score_col].transform(
            lambda x: x.rolling(window, min_periods=1).max()
        )

# Momentum (change / acceleration)
for risk_type in RISK_CATEGORIES:
    score_col = f"climate_risk_{risk_type}_score"
    _tmp[f"climate_risk_{risk_type}_change_1d"] = _tmp.groupby("region_id")[score_col].diff(1)
    _tmp[f"climate_risk_{risk_type}_change_7d"] = _tmp.groupby("region_id")[score_col].diff(7)
    _tmp[f"climate_risk_{risk_type}_acceleration"] = _tmp.groupby("region_id")[f"climate_risk_{risk_type}_change_1d"].diff(1)

# Country aggregations
for risk_type in RISK_CATEGORIES:
    score_col = f"climate_risk_{risk_type}_score"
    weighted_col = f"climate_risk_{risk_type}_weighted"
    country_agg = _tmp.groupby(["country_name", "date_on"]).agg({
        score_col: ["mean", "max", "std"],
        weighted_col: "sum",
        "percent_country_production": "sum",
    }).round(4)
    country_agg.columns = [f"country_{risk_type}_{'_'.join(col).strip()}" for col in country_agg.columns]
    country_agg = country_agg.reset_index()
    _tmp = _tmp.merge(country_agg, on=["country_name", "date_on"], how="left")

futures_cols = [c for c in _tmp.columns if c.startswith("futures_")]
_tmp = _tmp.dropna(subset=futures_cols)
valid_ids = _tmp.dropna()["ID"].astype(str).unique()

print("valid_ids:", len(valid_ids))

if "ID" in df.columns:
    df["ID"] = df["ID"].astype(str)
    df = df[df["ID"].isin(valid_ids)].copy()

print("after align:", df.shape)



valid_ids: 219161
after align: (219161, 369)


In [ ]:
# Generate submission.csv

REQUIRED_COLS = [c for c in ["ID", "date_on", "country_name", "region_name"] if c in df.columns]

submit_climate_cols = [c for c in selected_feats if c.startswith("climate_risk_")]

futures_cols = [c for c in df.columns if c.startswith("futures_")]

if submit_climate_cols:
    df[submit_climate_cols] = df[submit_climate_cols].fillna(0)

submit_cols = REQUIRED_COLS + submit_climate_cols + futures_cols
submission = df[submit_cols].copy()

out_path = WORK_DIR / "submission.csv"
submission.to_csv(out_path, index=False)

print("submission shape:", submission.shape)
print("saved:", out_path)
print("nulls:", int(submission.isna().sum().sum()))

submission shape: (219161, 25)
saved: /kaggle/working/submission.csv
nulls: 0
